In [ ]:
# setup API keys

import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAIA_API_KEY"] = getpass.getpass("Enter OpenAI API key:")

from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider = "openai")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from typing import Sequence
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str
    
# define new graph
workflow = StateGraph(state_schema=State)

# define the function that calls the model
def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}

# define nodes in graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# invoke app
config = {"configurable": {"thread_id": "123"}}
query = "Hi, I'm Travis!"
language = "Italian"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages, "language": language}, config)
output["messages"][-1].pretty_print()

query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()
